In [1]:
import numpy as np

import pymc as pm
import aesara
import aesara.tensor as at

import arviz as az

WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
print(pm.__version__)
print(aesara.__version__)

4.1.4
2.7.9


In [3]:
c, sigma = 10., 1.5
X = np.random.normal(size=(100, 2))
beta = np.asarray([-3., 4.])
y = X @ beta + c + np.random.normal(0, sigma, size=100)

In [4]:
with pm.Model() as m:
    i = pm.Normal('c', 0, 100)
    β = pm.Normal('β', 0, 100, size=2)
    σ = pm.HalfNormal('σ', 10)
    obs = pm.Normal('y', X @ β + i, σ, observed=y)
    # trace = pm.sample(1000)
# az.plot_trace(trace);

In [5]:
init_point = m.initial_point()

val_and_grad = m.logp_dlogp_function()
val_and_grad.set_extra_values({})
q = pm.blocking.DictToArrayBijection.map({v.name: init_point[v.name] for v in m.value_vars})
val_and_grad(q)  # val_and_grad(q.data)

(array(-403.07851477),
 array([10.11309994, -3.25584444,  3.55693003, 27.25606927]))

In [6]:
# same output as:
m.compile_logp()(init_point), m.compile_dlogp()(init_point)

(array(-403.07851477),
 array([10.11309994, -3.25584444,  3.55693003, 27.25606927]))

In [7]:
hessian = m.d2logp()
vars = pm.aesaraf.cont_inputs(hessian)
value_var = at.concatenate([at.flatten(v) for v in vars], axis=0)
# value_var = [m.rvs_to_values.get(var) for var in m.free_RVs]
hvp = m.compile_fn(hessian @ value_var)
hvp(init_point)

array([ 46.57254631, -14.99371772,  16.38026813, 590.64102638])

In [8]:
# same output as:
m.compile_d2logp()(init_point) @ q.data

array([ 46.57254631, -14.99371772,  16.38026813, 590.64102638])